In [1]:
import mnist_loader
import network
import numpy as np

In [5]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

net = network.Network([784, 30, 10], cost=network.CrossEntropyCost)
net.large_weight_initializer()
net.SGD(training_data, 10, 10, 0.5, evaluation_data=test_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 9176 / 10000

Epoch 1 training complete
Accuracy on evaluation data: 9305 / 10000

Epoch 2 training complete
Accuracy on evaluation data: 9306 / 10000

Epoch 3 training complete
Accuracy on evaluation data: 9412 / 10000

Epoch 4 training complete
Accuracy on evaluation data: 9427 / 10000

Epoch 5 training complete
Accuracy on evaluation data: 9437 / 10000

Epoch 6 training complete
Accuracy on evaluation data: 9425 / 10000

Epoch 7 training complete
Accuracy on evaluation data: 9401 / 10000

Epoch 8 training complete
Accuracy on evaluation data: 9460 / 10000

Epoch 9 training complete
Accuracy on evaluation data: 9452 / 10000



([], [9176, 9305, 9306, 9412, 9427, 9437, 9425, 9401, 9460, 9452], [], [])

In [12]:
foo = net.feedforward(test_data)
np.shape(test_data[10])

ValueError: setting an array element with a sequence.